# EP1 - Conjunto de Maldelbrot e paralelização com pThreads e OpenMP

| Nome | NUSP |
|------|------|
| Giulia C. de Nardi | 10692203 |
| Vitor D. Tamae | 10705620 |
| Lucy Anne de Omena Evangelista | 11221776 |
| Leonardo Costa Santos | 10783142 |
| Alexandre Muller Jones | 8038149 |


INFORMAÇÔES SOBRE O DESENVOLVIMENTO DO RELATÓRIO:

- **A função run_experiments pode ser implementada a fim de fazer os 4 tipos de figuras se necessário.**

- **Não implementei isso pois não sei como será a plotagem dos gráficos. Se facilitar o processo de plotagem, implementamos essa parte.**

- **De forma geral, conseguimos realizar os experimentos das funções mandelbrot_seq, mandelbrot_omp e mandelbrot_pth contanto que haja na saída das funções a duração da execução, e como entrada da função o número de threads.**

- **Para o caso dos mandelbrot paralelizados, podemos ter dois tempos como saída: o considerando o tempo de alocação de memória e I/O, e o tempo não considerando esses fatores. Ainda não implementado para a realização das funções, mas de fácil ajuste nas funções feitas neste notebook.**

- **Ainda falta implementar as função de plotagem dos pontos.**

- **Para as sessões de Mandelbrot com PThreads e com OpenMP falta mudar o nome das funções. Por hora elas estão dando problema porque não há saída alguma das funções.**

## Configuração do ambiente

Atualizando os pacotes Julia

In [1]:
] up

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]


Verificando o status dos pacotes, e se há algum problema, com o comando:

In [5]:
] st

    Status `~/.julia/environments/v1.3/Project.toml`
  [336ed68f] CSV v0.6.2
  [a93c6f00] DataFrames v0.21.0
  [31c24e10] Distributions v0.23.2
  [7073ff75] IJulia v1.21.2
  [b964fa9f] LaTeXStrings v1.1.0
  [8314cec4] PGFPlotsX v1.2.6
  [1a8c2f83] Query v0.12.2
  [f3b207a7] StatsPlots v0.14.6


## Funções para realização dos experimentos

In [53]:
; make mandelbrot_seq

cc     mandelbrot_seq.c   -o mandelbrot_seq


In [54]:
; ./mandelbrot_seq

usage: ./mandelbrot_seq c_x_min c_x_max c_y_min c_y_max image_size
examples with image_size = 11500:
    Full Picture:         ./mandelbrot_seq -2.5 1.5 -2.0 2.0 11500
    Seahorse Valley:      ./mandelbrot_seq -0.8 -0.7 0.05 0.15 11500
    Elephant Valley:      ./mandelbrot_seq 0.175 0.375 -0.1 0.1 11500
    Triple Spiral Valley: ./mandelbrot_seq -0.188 -0.012 0.554 0.754 11500


In [55]:
; ./mandelbrot_seq 0.175 0.375 -0.1 0.1 200 0

3.485450

A função abaixo recebe parâmetros `size`, com o tamanho da imagem, `f`, com a id do tupo de fractal a ser feito (0 - Full, 2 - Triple Spiral, 3 - Elephant, 4 - Seahorse), `mandel`, com o nome da função a ser executada (`./mandelbrot_seq`, `./mandelbrot_opm`, `./mandelbrot_pth`),e `threads`, com o número de threads do programa paralelo. A função executa o programa `mandelbrot` com os parâmetros dados e devolve um `DataFrame` com os resultados.

**#precisamos colocar a duração do programa como uma saida da funcao pra conseguir fazer as estimativas.**

**#também precisamos colocar a quantidade de threads como uma entrada do programa que usa #openMP e pthreads**

**#tudo feito daqui pra baixo leva em conta que a duração do programa é uma das saídas**

In [193]:
using DataFrames, Query, StatsPlots, Statistics

function measure_mandelbrot(size, f, mandel; thread = 0)    
    if f == 0  mode = `-2.5 1.5 -2.0 2.0`
    elseif f == 1  mode = `-0.188 -0.012 0.554 0.754`
    elseif f == 2  mode = `0.175 0.375 -0.1 0.1`
    elseif f == 3  mode = `-0.8 -0.7 0.05 0.15`
    end
    
    if thread != 0 
    results = parse.(Float64,
        read(`./$mandel $mode $size $thread`, String))
    else
    results = parse.(Float64,
        read(`./$mandel $mode $size`, String))
    end
        
    return DataFrame(size = size,
        f = f,
        threads = thread,
        duration = results[1])
end

measure_mandelbrot (generic function with 1 method)

A função `run_experiments` recebe os mesmos parâmetros `size`, `f`,`mandel` e `threads`, e um parâmetro adicional `repetitions`, com o número de repetições de cada experimento com um dado número de `threads`. A função devolve um `DataFrame` com todos os experimentos.

In [197]:
function run_experiments(size, f, mandel, repetitions; threads = [])
    run(`make $mandel`)
    
    results = DataFrame(size = Int[],
        f = Int[],
        threads = Int[],
        duration = Float64[]) 
    
    if threads != []
    for t in threads
        for s in size
        for r in 1:repetitions
            append!(results,
                measure_mandelbrot(s, f, mandel, thread = t))    
        end
        end
    end
    else
        for r in 1:repetitions
        for s in size
            append!(results,
                measure_mandelbrot(s, f, mandel))    
        end
        end
    end 
    
    return results
end

run_experiments (generic function with 1 method)

A função `parse_results` recebe um `DataFrame` de resultados, produzido pela função `run_experiments`. A função devolve um `DataFrame` com a média e o intervalo de confiança da média a 95% dos tempos de execução, agrupados por número de threads.

In [195]:
function parse_results(results)
    parsed_results = results |>
                    @groupby({_.threads,_.size}) |>
                    @map({threads = key(_).threads,
                          size = _.size[1],
                          mean_duration = mean(_.duration),
                          ci_duration = 1.96 * std(_.duration)}) |>
                    DataFrame
    
    return parsed_results
end

parse_results (generic function with 1 method)

## Funções para traçar gráficos

In [204]:
# espaço para funções

## Mandelbrot sequencial

Nesta parte, traremos a execução do maldelbrot em sua versão sequencial, junto com a análise de tempo de execução para os diferentes tipo de fractais Triple Spiral, Elephant, Seahorse & Full) e em diferentes resoluções ($ 2^4 \cdots 2^{13}$)

Realizando as medições para o mandelbrot sequencial:

In [200]:
size = [2 ^ x for x in 4:13]
f = 0
thread = [2 ^ x for x in 0:5]
repetitions = 10

results = run_experiments(size, f, "mandelbrot_seq", repetitions)
parsed_results = parse_results(results)

make: 'mandelbrot_seq' is up to date.


,threads,size,mean_duration,ci_duration
,Int64,Int64,Float64,Float64
1,0,8,3.0,0.0
2,0,16,3.0,0.0
3,0,32,3.0,0.0
4,0,64,3.0,0.0
5,0,128,3.0,0.0
6,0,256,3.0,0.0


In [ ]:
size = [2 ^ x for x in 4:13]
f = 1
thread = [2 ^ x for x in 0:5]
repetitions = 10

results = run_experiments(size, f, "mandelbrot_seq", repetitions)
parsed_results = parse_results(results)

In [ ]:
size = [2 ^ x for x in 4:13]
f = 2
thread = [2 ^ x for x in 0:5]
repetitions = 10

results = run_experiments(size, f, "mandelbrot_seq", repetitions)
parsed_results = parse_results(results)

In [ ]:
size = [2 ^ x for x in 4:13]
f = 3
thread = [2 ^ x for x in 0:5]
repetitions = 10

results = run_experiments(size, f, "mandelbrot_seq", repetitions)
parsed_results = parse_results(results)

## Mandelbrot com pthreads

In [ ]:
size = [2 ^ x for x in 4:13]
f = 0
thread = [2 ^ x for x in 0:5]
repetitions = 10

results = run_experiments(size, f, "mandelbrot_seq", repetitions)
parsed_results = parse_results(results)

In [ ]:
size = [2 ^ x for x in 4:13]
f = 1
thread = [2 ^ x for x in 0:5]
repetitions = 10

results = run_experiments(size, f, "mandelbrot_seq", repetitions)
parsed_results = parse_results(results)

In [ ]:
size = [2 ^ x for x in 4:13]
f = 2
thread = [2 ^ x for x in 0:5]
repetitions = 10

results = run_experiments(size, f, "mandelbrot_seq", repetitions)
parsed_results = parse_results(results)

In [ ]:
size = [2 ^ x for x in 4:13]
f = 3
thread = [2 ^ x for x in 0:5]
repetitions = 10

results = run_experiments(size, f, "mandelbrot_seq", repetitions)
parsed_results = parse_results(results)

## Mandelbrot com OpenMP

In [203]:
size = [2 ^ x for x in 4:13]
f = 0
thread = [2 ^ x for x in 0:5]
repetitions = 10

results = run_experiments(size, f, "mandelbrot_seq", repetitions)
parsed_results = parse_results(results)

make: 'mandelbrot_seq' is up to date.


,threads,size,mean_duration,ci_duration
,Int64,Int64,Float64,Float64
1,0,16,3.0,0.0
2,0,32,3.0,0.0
3,0,64,3.0,0.0
4,0,128,3.0,0.0
5,0,256,3.0,0.0
6,0,512,3.0,0.0
7,0,1024,3.0,0.0
8,0,2048,3.0,0.0
9,0,4096,3.0,0.0


In [ ]:
size = [2 ^ x for x in 4:13]
f = 1
thread = [2 ^ x for x in 0:5]
repetitions = 10

results = run_experiments(size, f, "mandelbrot_seq", repetitions)
parsed_results = parse_results(results)

In [ ]:
size = [2 ^ x for x in 4:13]
f = 2
thread = [2 ^ x for x in 0:5]
repetitions = 10

results = run_experiments(size, f, "mandelbrot_seq", repetitions)
parsed_results = parse_results(results)

In [ ]:
size = [2 ^ x for x in 4:13]
f = 3
thread = [2 ^ x for x in 0:5]
repetitions = 10

results = run_experiments(size, f, "mandelbrot_seq", repetitions)
parsed_results = parse_results(results)